In [ ]:
import hvplot.pandas
import geopandas as gpd
import numpy as np
import pandas as pd
import pyproj
import seareport_data as D
import shapely

In [ ]:
# First let's get the OSM dataset
# - wgs84_area is the area of the polygons calculated on the WGS84 **ellipsoid**
# - wgs84_perimeter is the area of the perimeter calculated on the WGS84 **ellipsoid**
# This means that they are accurate value regardless of the CRS
OSM = D.osm_df()
OSM

In [ ]:
# Let's also get the UTM tiles
UTM = D.utm_df()
UTM

In [ ]:
# now let's pick one island to demonstrate how concave hull works
#
# Tip to find the FID of a specific island, load the dataset on QGIS and use the "Idenfity Features Button": Ctrl + Shift + I
# To find the path to the dataset, you can use:
# D.osm()[0]
#
# So, we will work with Limnos: 
island = OSM.iloc[[36593]]
island
island.hvplot.polygons(tiles=True, alpha=0.5)

In [ ]:
# Now, the latest GEOS versions support concave hulls. 
# This bindings for it has been implemented on shapely/geopandas, too
# https://shapely.readthedocs.io/en/2.0.6/reference/shapely.concave_hull.html
# https://geopandas.org/en/latest/docs/reference/api/geopandas.GeoSeries.concave_hull.html
# Let's see how it works
island.concave_hull().to_frame("geometry").hvplot.polygons(tiles=True, alpha=0.5)

In [ ]:
# That's pretty horrible... What are these lines?
# Let's see if we can tweak it:
island.concave_hull?

In [ ]:
# Hmm. there is this `ratio` which seems to be useful. 
# Nevertheless it will not help in our case (we will still use it later on, but it will not solve this specific problem).
# 
# The problem with this concave hull algorithm is that it is supposed to work with 
# a "soup" of points thar are more or less uniformly distributed.
# More specifically, the algorithm is doing a Delauney triangulation on all the points
# and it starts to remove the outer triangles
#
# The problem is that ALL our points are on the boundary of the polygon and there are no points inside it.
# So the triangles are few and they are huge, too. 
# 
# That's the problem
# But it can be solved. We just need to add triangles inside. 
# Let's do that. geopandas has the right method for this:
island.sample_points?

In [ ]:
# So for example:
island.sample_points(size=500, rng=42).to_frame("geometry").hvplot.points(tiles=True)

In [ ]:
# Hmm... that seems to work well enough. 
# The appropriate `size` needs to be chosen for each polygon separately, though,
# but it shouldn't be too hard to find an appropriate formula
def concave_hull(gdf, ratio):
    no_points = np.maximum(gdf.no_coords, np.minimum(10_000, (gdf.wgs84_area // 10_0000).astype(int)))
    inner = gdf.sample_points(no_points)
    outer = gdf.boundary.apply(lambda g: shapely.MultiPoint(g.coords))
    #outer = gdf.points
    total = inner.union(outer)
    a = total.concave_hull(ratio=ratio).union(gdf).union_all()
    out = gpd.GeoDataFrame(
        geometry=[total.concave_hull(ratio=ratio).union(gdf).union_all()], 
        crs=gdf.crs,
    )
    out = out.explode().exterior.polygonize().to_frame("geometry")
    return out

In [ ]:
# For the record, we probably need to do the concave hull calculations in projected space, but for demonstration purposes it should be fine to stay on 4326

In [ ]:
concave_hull(island, 0.1).hvplot(tiles=True, alpha=0.5)

In [ ]:
# Hey! that seemed to work!!!
# Let's see now what `ratio` does
concave_hull(island, 0.00).hvplot(tiles=True, alpha=0.5)
concave_hull(island, 0.05).hvplot(tiles=True, alpha=0.5)
concave_hull(island, 0.10).hvplot(tiles=True, alpha=0.5)
concave_hull(island, 0.30).hvplot(tiles=True, alpha=0.5)

In [ ]:
# So, the higher ratio, the more convex the curve.
# And if ratio=1.0 then we get the convex hull:
concave_hull(island, 1).hvplot(tiles=True, alpha=0.5)
island.convex_hull.to_frame("geometry").hvplot(tiles=True, alpha=0.5)

In [ ]:
# So in a nutshell our problem is:
# 
# For each polygon:
#   1. Figure out an appropriate `size` for `sample_points()`. 
#   2. Figure out an appropriate `ratio` for `concave_hull()`. 
# 
# The first problem is not that difficult. Too few points means bad triangles. 
# Too many points means slow calculations. 
# So we can always just use more points than strictly necessary
#
# The second problem is more challenging though. If we test with more islands we will see that
# the ratio value depends on the area of the polygon, the number of points and the 
# distribution of the points (shape of the island).
# 
# Generally speaking, bigger islands need smaller ratios compared to smaller islands (in order to produce similar concave hulls).

In [ ]:
# Quantifying concave hulls
# These are some ways we could try to comparing one concave hull Polygon with a different one:
# 
# concave_hull_area / original_area
# concave_hull_perimeter / original_perimeter
# concave_hull_no_points / original_no_points
#
# To calculate these ratios we will need a help function

def calc_area_and_perimeter(
    gdf: gpd.GeoDataFrame,
    geod: pyproj.Geod = pyproj.Geod(ellps="WGS84"),
    *,
    sort: bool = False,
) -> gpd.GeoDataFrame:
    assert gdf.crs is not None, "CRS must be defined!"
    area_and_perimeter = gdf.to_crs(4326).geometry.apply(geod.geometry_area_perimeter).apply(pd.Series)
    area_and_perimeter = area_and_perimeter.rename(columns={0: "area", 1: "perimeter"})
    area_and_perimeter.area *= -1
    gdf = gdf.assign(
        wgs84_area=area_and_perimeter.area,
        wgs84_perimeter=area_and_perimeter.perimeter,
        no_coords=gdf.count_coordinates() - 1,
    )
    if sort:
        gdf = gdf.sort_values("wgs84_area", ascending=False).reset_index(drop=True)
    return gdf

concave_0000 = calc_area_and_perimeter(concave_hull(island, 0.00))
concave_0050 = calc_area_and_perimeter(concave_hull(island, 0.05))
concave_0100 = calc_area_and_perimeter(concave_hull(island, 0.10))
concave_1000 = calc_area_and_perimeter(island.convex_hull.to_frame("geometry")).reset_index(drop=True)
pd.concat([concave_0000, concave_0050, concave_0100, concave_1000]).drop(columns=["geometry"]).divide(island.drop(columns=["geometry", "fid"]).reset_index(drop=True), axis=1)

In [ ]:
# here are the raw numbers
pd.concat([island, concave_0000, concave_0050, concave_0100, concave_1000])

In [ ]:
# Apart from that we can also use the hausdorf distance
# (i.e. the maximum distance of the closest nodes): https://en.wikipedia.org/wiki/Hausdorff_distance
#
# In a nutshell, the hausdorff distance is the maxium distance between an island node and its closest mesh node.
# 
# Since the calculation of the hausdorff distnace needs to happen on a Projected CRS we will use the appropriate UTM zone.
island_32635 = island.to_crs(32635)
float(concave_0000.to_crs(32635).hausdorff_distance(island_32635, align=False).iloc[0])
float(concave_0050.to_crs(32635).hausdorff_distance(island_32635, align=False).iloc[0])
float(concave_0100.to_crs(32635).hausdorff_distance(island_32635, align=False).iloc[0])
float(concave_1000.to_crs(32635).hausdorff_distance(island_32635, align=False).iloc[0])